In [1]:
import numpy as np
import networkx as nx
import Graph_utils as gu

In [2]:
#Graph Construction

G = nx.Graph()
text_corpus, file_names, target = gu.read_data()
cosine_sim_matrix = gu.get_cosine_sim_matrix(text_corpus)
edge_list = gu.get_edge_list(cosine_sim_matrix,file_names)
G.add_weighted_edges_from(edge_list)
G = gu.remove_self_loops(G)


In [3]:
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 600
Number of edges: 179700
Average degree: 599.0000


In [4]:
#Stage 1 -  Random Surfing


In [5]:
#Stage 2 - PPMI Matrix


In [6]:
#Stage 3 - Stacked De-noising AutoEncoder